<a href="https://colab.research.google.com/github/PeKkaPie95/selfProjectsAndAttempts/blob/main/csiLossFunc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch==2.0.0+cu117
!pip install pytorch-lightning==1.9.4
!pip install accelerate==0.21.0
!pip install tokenizers==0.13.3
!pip install transformers==4.26.1
!pip install torchmetrics


ERROR: Could not find a version that satisfies the requirement torch==2.0.0+cu117 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1)
ERROR: No matching distribution found for torch==2.0.0+cu117


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchmetrics.classification import BinaryConfusionMatrix

class CriticalSuccessIndexMetric:
    def __init__(self):
        self.confusion_matrix = BinaryConfusionMatrix()

    def update(self, preds, targets):
        # Convert predictions to binary labels if they are probabilities
        preds = (preds > 0.5).int()  # Threshold at 0.5
        targets = targets.int()
        self.confusion_matrix.update(preds, targets)

    def compute(self):
        cm = self.confusion_matrix.compute()
        tp = cm[1, 1].item()  # True Positives
        fp = cm[0, 1].item()  # False Positives
        fn = cm[1, 0].item()  # False Negatives
        csi = tp / (tp + fp + fn + 1e-6)  # Avoid division by zero
        return csi

    def reset(self):
        self.confusion_matrix.reset()

In [ ]:
class CSILoss(nn.Module):
    def __init__(self):
        super(CSILoss, self).__init__()

    def forward(self, predictions, targets):
        predictions = torch.clamp(predictions, 1e-6, 1 - 1e-6)  # Clipped to avoid 0 and 1
        predicted_labels = (predictions > 0.5).float()

        TP = (predicted_labels * targets).sum()
        FP = (predicted_labels * (1 - targets)).sum()
        FN = ((1 - predicted_labels) * targets).sum()

        csi = TP / (TP + FP + FN + 1e-6)

        # Convert CSI to a tensor with requires_grad=True
        csi = torch.tensor(csi, requires_grad=True) # This line is added to make CSI part of the computation graph

        return (1 - csi).float()  # Return 1 - CSI as loss
        # this line was cause of an error which took way to long to fix
        # also the above condition of required_grad to be true
        # It converts the calculated csi value (which is a Python float) into a PyTorch tensor
        # and explicitly sets requires_grad=True.
        # This ensures that csi becomes part of the computation graph,
        # allowing gradients to be calculated for it during backpropagation.

In [ ]:
class BinaryClassifier(nn.Module):
    def __init__(self, input_dim):
        super(BinaryClassifier, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()  # Output as probability (between 0 and 1)
        )

    def forward(self, x):
        return self.fc(x).float()

In [ ]:
def train_binary_classifier_with_metrics():

    # Sample data
    torch.manual_seed(42)
    X = torch.rand(100, 10)  # 100 samples, 10 features
    y = torch.randint(0, 2, (100, 1)).float()  # Binary targets

    model = BinaryClassifier(input_dim=10)
    criterion = CSILoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    # Ensure model parameters require gradients
    for param in model.parameters():
        assert param.requires_grad == True  # Check if parameters require gradients

    csi_metric = CriticalSuccessIndexMetric()

    epochs = 50
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        csi_metric.reset()  # Reset CSI metric at the start of each epoch

        outputs = model(X)  # Forward pass, outputs connected to model parameters
        loss = criterion(outputs, y)  # Calculate loss
        loss.backward()  # Backpropagation to compute gradients

        optimizer.step()  # Update model parameters

        csi_metric.update(outputs, y)  # Update CSI metric

        if (epoch + 1) % 5 == 0:
            csi = csi_metric.compute()  # Compute CSI
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, CSI: {csi:.4f}")


In [ ]:
if __name__ == "__main__":
    train_binary_classifier_with_metrics()

<ipython-input-29-442c80e6a386>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  csi = torch.tensor(csi, requires_grad=True) # This line is added to make CSI part of the computation graph


Epoch [5/50], Loss: 0.9756, CSI: 0.0244
Epoch [10/50], Loss: 0.9756, CSI: 0.0244
Epoch [15/50], Loss: 0.9756, CSI: 0.0244
Epoch [20/50], Loss: 0.9756, CSI: 0.0244
Epoch [25/50], Loss: 0.9756, CSI: 0.0244
Epoch [30/50], Loss: 0.9756, CSI: 0.0244
Epoch [35/50], Loss: 0.9756, CSI: 0.0244
Epoch [40/50], Loss: 0.9756, CSI: 0.0244
Epoch [45/50], Loss: 0.9756, CSI: 0.0244
Epoch [50/50], Loss: 0.9756, CSI: 0.0244
